# Alzheimer & MCI DeepLearning(CNN) Classifier
- ADNI MRI 이미지로 알츠하이머와 경도인지장애를 분류하는 딥러닝 프로젝트
- 데이터 출처
  - 메타데이터 : https://ida.loni.usc.edu/
  - MRI이미지 : https://www.kaggle.com/datasets/katalniraj/adni-extracted-axial
- 저작권에 의해 데이터는 깃허브에 업로드 하지 않았음

## Part2-3. CNN Hyperparameter Tuning
- 본 작업은 Google Colaboratory(Colab)에서 진행되었음.
  - Python version : 3.8
  - Tensorflow version : 2.9

### 1. Library & Data Import

- Library Import

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['axes.unicode_minus'] = False
import seaborn as sns

import tensorflow as tf
from keras.utils import image_dataset_from_directory
from keras.models import Sequential
from keras.layers import Resizing, Rescaling, RandomFlip, RandomRotation
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.callbacks import Callback
import IPython

In [2]:
!pip install -U keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import keras_tuner as kt

- Google Drive Mount

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


- Directory 지정 & 폴더 확인

In [5]:
base_dir = "/content/drive/MyDrive/AI/Alzheimer/"
data_dir = base_dir + "Axial"
os.listdir(data_dir)

['CN', 'AD', 'MCI']

- Global 변수 지정
  - 이전 노트와 동일하게 설정
  - 클래스 순서, 이미지 크기(256), 배치사이즈(32), 채널(3), 에포크(10), 시드(42)

In [6]:
CLASSES = ['CN', 'MCI', 'AD']
IMG_SIZE = 256
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 10
SEED = 42

- Data 불러오기

In [7]:
dataset = image_dataset_from_directory(
    data_dir,
    shuffle=True,
    class_names=CLASSES,
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE,IMG_SIZE),
    seed=SEED
)
print(dataset.class_names)

Found 5154 files belonging to 3 classes.
['CN', 'MCI', 'AD']


### 2. Data Split & Prefetch

- Data Split
  - Train : Validation : Test = 0.8 : 0.1 : 0.1
  - Batch 단위로 묶인 데이터라 Batch단위로 Split

In [8]:
def dataset_split(ds, tr=0.8, val=0.1, test=0.1, shuffle=True, buf_size=10000, SEED=42):
  ds_size = len(ds)

  if shuffle:
    ds = ds.shuffle(buf_size,seed=SEED)
  train_size = int(ds_size*tr)
  val_size = int(ds_size*test)
  
  train = ds.take(train_size)
  test0 = ds.skip(train_size)
  val = test0.take(val_size)
  test = test0.skip(val_size)

  return train,val,test

train_ds, val_ds, test_ds = dataset_split(dataset)
print("Split전 Batched data 개수")
print(f"Dataset : {len(dataset)}")
print("\nSplit후 Batched data 개수")
print(f"Train : {len(train_ds)}")
print(f"Validation : {len(val_ds)}")
print(f"Test : {len(test_ds)}")

Split전 Batched data 개수
Dataset : 162

Split후 Batched data 개수
Train : 129
Validation : 16
Test : 17


- Data Prefetch
  - 속도를 향상시키기 위한 목적

In [9]:
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds=val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

### 3. Model Building (Keras-Tuner)
- Keras-Tuner를 이용해 튜닝 진행할 것임
  - Tuning Method : Hyperband
- 정규화
  - Resizing : (256,256)으로 크기 맞춤
  - Rescaling : 0~255의 데이터를 0~1로 정규화
- 데이터 증강 (Augmentation) - Train dataset에만 적용되는 layer
  - RandomFlip : 수직, 수평으로 데이터를 뒤집어 훈련데이터를 증강
  - RandomRotation : 0.1정도를 Random하게 기울여 훈련데이터를 증강
- 튜닝할 하이퍼파라미터
  - 데이터 특징 추출 layer (Feature Extractor)
    - Conv2D 차원수 (filters) : 16, 32, 64
    - 2 Block 모두 동일한 filters 적용
  - 데이터 분류 layer (Classifier)
    - 은닉층 노드 수 (Dense units) : 64 ~ 512, 간격 64
- 모델 컴파일(compile)
  - Optimizer : Adam
  - 손실함수 (Loss function) : Sparse Categorical Crossentropy
  - 평가지표 (Metrics) : Accuracy 

In [10]:
# 정규화 및 증강 layer 정의
resize_rescale = Sequential([
    Resizing(IMG_SIZE, IMG_SIZE),
    Rescaling(1.0/255)
])
data_augmentation = Sequential([
    RandomFlip("horizontal_and_vertical", seed=SEED),
    RandomRotation(0.1, seed=SEED)
])

- Model builder 함수 정의

In [11]:
INPUT_SHAPE = (BATCH_SIZE, IMG_SIZE, IMG_SIZE, CHANNELS)

def model_builder(hp):
  model = Sequential()
  model.add(resize_rescale)
  model.add(data_augmentation)

  hp_conv2d = hp.Choice('filters', values = [16, 32, 64])
  model.add(Conv2D(filters=hp_conv2d, kernel_size=(3,3), padding='same', activation='relu', input_shape=INPUT_SHAPE))
  model.add(MaxPooling2D((2,2)))
  model.add(Conv2D(filters=hp_conv2d, kernel_size=(3,3), padding='same', activation='relu'))
  model.add(MaxPooling2D((2,2)))

  model.add(Flatten())

  hp_units = hp.Int('units', min_value = 64, max_value = 512, step = 64)
  model.add(Dense(units=hp_units, activation='relu'))
  model.add(Dense(3, activation='softmax'))

  model.compile(optimizer='adam',
                loss = 'sparse_categorical_crossentropy',
                metrics=['acc'])
  return model

- Tuner 정의
  - 성능 개선 여부 판단 기준 : Validation loss
  - 최대 epochs : 10

In [12]:
tuner = kt.Hyperband(model_builder,
                     objective='val_acc',
                     max_epochs=10,
                     factor=3,
                     directory=base_dir,
                     project_name='kt_hyperband')

- 이전 출력을 지우는 Callback 정의

In [13]:
class ClearTrainingOutput(Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait=True)

### 4. Hyperparameter Tuning 진행

In [14]:
tuner.search(train_ds, epochs=10, validation_data = val_ds, callbacks=[ClearTrainingOutput()])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 29 Complete [00h 02m 41s]
val_acc: 0.912109375

Best val_acc So Far: 0.98046875
Total elapsed time: 00h 47m 34s


- 튜닝 진행 결과

In [15]:
print(f"""
하이퍼 파라미터 검색 완료. 
최적화된 Conv2D 필터 수 : {best_hps.get('filters')}
최적화된 은닉층 Dense 노드 수 : {best_hps.get('units')}
""")


하이퍼 파라미터 검색 완료. 
최적화된 Conv2D 필터 수 : 16
최적화된 은닉층 Dense 노드 수 : 320



### 5. Best Parameter Modeling

- 최적화된 하이퍼파라미터
  - Conv2D 필터 수 : 16
  - Classifier 은닉층 Dense 노드 수 : 320

In [18]:
model = tuner.hypermodel.build(best_hps)
model.build(INPUT_SHAPE)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 256, 256, 3)       0         
                                                                 
 sequential_1 (Sequential)   (None, 256, 256, 3)       0         
                                                                 
 conv2d_4 (Conv2D)           (32, 256, 256, 16)        448       
                                                                 
 max_pooling2d_4 (MaxPooling  (32, 128, 128, 16)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (32, 128, 128, 16)        2320      
                                                                 
 max_pooling2d_5 (MaxPooling  (32, 64, 64, 16)         0         
 2D)                                                  

### 6. Model Export (모델 저장, 부호화)

- 튜닝 이후 GPU 할당량 초과로 인해 이후 최종 모델 학습은 로컬에서 진행할 것임
- 튜닝 후 최적화된 하이퍼파라미터가 적용된 모델 저장

In [19]:
model.save(filepath="/content/drive/MyDrive/AI/Alzheimer/model_tuned.hdf5")

## Part3 ~

- GPU 할당량 초과로 더 이상 Colab에선 진행 할 수 없었음...
- 최종모델 학습 및 성능 비교 분석은 Local 환경에서 진행

- Part3-1. Final model Training & Evaluation
  - 최종 모델 학습 & 검증
- Part3-2. Plotting & Prediction
  - 튜닝 전후 모델 시각화 및 비교분석